In [1]:
import numpy as np
import cv2
import os
from os.path import isfile, join
import matplotlib.pyplot as plt
%matplotlib inline

1. Video ↔ Images: Write a program to convert a given video to its constituent
images. Your output should be in a specified folder. Write another program that
will merge a set of images in a folder into a single video. You should be able to
control the frame rate in the video that is created

In [2]:
def getFrames(videofile,frameRate):
    vidcap = cv2.VideoCapture(videofile)
    sec = 0
    count=0
    success = True
    while success:
        vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
        success,image = vidcap.read()
        if success:
            cv2.imwrite("op/image"+str(count)+".jpg", image)     # save frame as JPG %%file
        count = count + 1
        sec = sec + frameRate
        sec = round(sec, 2)
        

In [3]:
getFrames('video.mp4',0.25)

In [4]:
def makeVideo(pathIn,pathOut,fps):
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
    files.sort(key = lambda x: int(x[5:-4]))
    frame_array = []

    for i in range(len(files)):
        filename=pathIn + files[i]
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        frame_array.append(img)
        
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()
    


In [5]:
makeVideo('./op/','opvideo.mp4',12)

2. Capturing Images: Learn how to capture frames from a webcam connected to
your computer and save them as images in a folder. You may use either the
built-in camera of your laptop or an external one connected through USB. You
should also be able to display the frames (the video) on the screen while
capturing

In [6]:
def getWebcamFrames(n):
    vidcap = cv2.VideoCapture(0)
    count=0
    success = True
    while success:
        success,image = vidcap.read()
        if success:
            cv2.imwrite("webop/image"+str(count)+".jpg", image)     # save frame as JPG file
        count = count + 1
        if count>=n:
            break
    cv2.destroyAllWindows()

In [7]:
getWebcamFrames(50)

3 . Chroma Keying:
Create an interesting composite of two videos using this technique.

In [8]:
def chromaKeyFrames(pathFront,pathBack,pathOut,fps,low,high):
    files = [f for f in os.listdir(pathFront) if isfile(join(pathFront, f))]
    files.sort(key = lambda x: int(x[5:-4]))
    frame_array = []
    
    for i in range(len(files)):
        image = cv2.imread(pathFront + files[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        height, width, layers = image.shape
        size = (width,height)
        lower_green = np.array(low) 
        upper_green = np.array(high) 
        
        mask = cv2.inRange(image, lower_green, upper_green)
        image[mask != 0] = [0, 0, 0]

        background_image = cv2.imread(pathBack + files[i])
        background_image = cv2.cvtColor(background_image, cv2.COLOR_BGR2RGB)
        background_image[mask == 0] = [0, 0, 0]

        # Add the two images together to create a complete image!
        final_image = cv2.cvtColor(background_image + image, cv2.COLOR_RGB2BGR)
        frame_array.append(final_image)
    
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for i in range(len(frame_array)):
        out.write(frame_array[i])
    out.release()

In [9]:
chromaKeyFrames('./front/','./back/','chromavideo.mp4',10,[0,200,0],[20,255,70])